# Classification Metrics
Author: Kim Hazed Delfino

## Imports

In [215]:
import pandas as pd
import numpy as np
# for creating confusion metrix heatmap
import matplotlib.pyplot as plt
import seaborn as sns 
# Necessary tools 
from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
# Score metrics 
from sklearn.metrics import accuracy_score, precision_score,classification_report, ConfusionMatrixDisplay, confusion_matrix
# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

#ignore warning when model fitted with feature_names unnused
import warnings
warnings.filterwarnings("ignore") 


## Load Data

In [216]:
data  = load_breast_cancer()
X = pd.DataFrame(data.data, columns= data.feature_names)
y = pd.DataFrame(data.target, columns=["outcome"])
print(y.value_counts(normalize=True))
X.head()

outcome
1          0.627417
0          0.372583
dtype: float64


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [217]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [218]:
# Split Train-test 
X_train , X_test , y_train , y_test = train_test_split(X, y, random_state=42)

In [219]:
class ClfMetrics:
    def __init__(self, model, X_train, X_test, y_train, y_test, 
                 labels):
        self.model = model
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test 
        self.labels = labels

        #Instantiate and train model 
        model_map = {
            "dt_clf" : DecisionTreeClassifier,
            # added baggin/rf models for score comparison
            "bag_clf" : BaggingClassifier,
            "rf_clf" : RandomForestClassifier
        }
        if model == "dummy_clf":
            model_clf = DummyClassifier(strategy="most_frequent")
        else:
            model_clf = model_map[self.model](random_state=42)
        scaler = StandardScaler()
        self.model_clf_pipe = make_pipeline(scaler,model_clf)
        self.model_clf_pipe.fit(X_train,y_train)
        # set prediction
        self.model_train_pred = self.model_clf_pipe.predict(X_train)
        self.model_test_pred = self.model_clf_pipe.predict(X_test)
        # set classfication report
        self.train_dt_report = classification_report(y_train,self.model_train_pred,
                                                target_names=labels)
        self.test_dt_report = classification_report(y_test, self.model_test_pred,
                                            target_names=labels)
        
    # Display Confusion Report 
    def get_train_report(self):
        return print(f"Train Metric Scores: \n {self.train_dt_report}")

    def get_test_report(self):    
        return print(f"Test Metric Scores: \n {self.test_dt_report}")

    def get_visuals(self, normalize = None):
        # Display Confusion Matrix
        fig, axes = plt.subplot(ncols= 2, figsize=(10,5))
        ConfusionMatrixDisplay.from_predictions(y_test, self.model_train_pred,
                                                ax=axes[0],display_labels=self.labels,
                                                cmap="Blues",normalize=normalize)
        axes[0].set_title("Test CM")
          
        ConfusionMatrixDisplay.from_predictions(y_test, self.model_test_pred,
                                                ax=axes[1],display_labels=self.labels,
                                                cmap="Reds",normalize=normalize)
        axes[1].set_title("Test CM")
        
        return plt.show()
        


- Which of these do you think is worse in this case? If we have to increase one kind of error in order to minimize the other kind, which would we want to minimize? Why?
    - In this dataset , I think getting False Negative (type2) prediction is more problematic since they are actually ill but did not receive any treatment

In [220]:
# Target names for breast cancer dataset 
bc_labels = ["Benign","Malignant" ]

#Instantiate DecisionTree Classifier Model
dt_clf = ClfMetrics("dt_clf", X_train , X_test , y_train , y_test, bc_labels)

In [221]:
# Instantiate Dummy Classifier strategy = most_frequent
dummy_clf = ClfMetrics("dummy_clf", X_train , X_test , y_train , y_test, labels)

In [222]:
# Display classification_reports on both models


NameError: name 'df' is not defined